In [14]:
import requests
import json
import pandas as pd
from datetime import timedelta, date, datetime
import time

addresses = set(
    [
        "0xef6d69922bc2d038cb508086846524f8011c4a74",
        "0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459",
        "0xe1d29d0a39962a9a8d2a297ebe82e166f8b8ec18",
        "0xd5a5c2ef8977ab40d920417018145508251a07c0",
        "0x1b2965dc3b1697dd10b7126ec7393d79cda2cf91",
        "0x78f0269f5b1ca914c8c58993f41181f89e212b27",
        "0x8dbbca57ea56290efa14d835bbfd34faf1d89753",
        "0x53dda0a69992b87e51f0f14b1337281f09a2c8da",
        "0x63a9dbce75413036b2b778e670aabd4493aaf9f3",
        "0x3612b2e93b49f6c797066ca8c38b7f522b32c7cb",
        "0xebf02c6e12c3ee119abba161c40bfeead0a06b15",
        "0xf6bc89cc57e33c4e8053c4d8e63b70bf8c47e1a1",
        "0x530cf036ed4fa58f7301a9c788c9806624cefd19",
        "0x79ecd68a6bcdf40f40bf0dc0c1fcc962aa8c6213",
        "0x6b67623ff56c10d9dcfc2152425f90285fc74ddd",
        "0xe10824fd1ffb9951215025eb26cda42d2d8d9c4c",
        "0x9e199d8a3a39c9892b1c3ae348a382662dcbaa12",
        "0xd5a9c4a92dde274e126f82b215fccb511147cd8e",
        "0xbff79922fcbf93f9c30abb22322b271460c6bebb",
        "0xb53349160e38739b37e4bbfcf950ed26e26fcb41",
    ]
)
events = []
a = None
url = "https://api.opensea.io/api/v1/events"
for address in addresses:
    querystring = {
        "only_opensea": "false",
        "offset": "0",
        "limit": "200",
        "account_address": address,
        "event_type": "successful",
    }
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    events += json.loads(response.text)["asset_events"]
    time.sleep(10)


In [33]:
buy_sell_dict = {"timestamp": [], "asset_name": [], "type": [], "price": []}
for event in events:
    if event["asset"] is None:
        continue
    buy_sell_dict["timestamp"].append(
        datetime.strptime(event["transaction"]["timestamp"], "%Y-%m-%dT%H:%M:%S")
    )
    buy_sell_dict["asset_name"].append(event["asset"]["asset_contract"]["name"])
    buy_sell_dict["type"].append(
        "sell" if event["winner_account"]["address"] not in addresses else "buy"
    )
    buy_sell_dict["price"].append(float(event["total_price"]) / 10e17)

buy_sell_df = pd.DataFrame.from_dict(buy_sell_dict)

buy_sell_df


,timestamp,asset_name,type,price
0,2021-09-06 00:43:19,Autoglyphs,sell,375.00
1,2021-08-30 18:38:59,Meebits,sell,9.00
2,2021-08-25 18:31:34,Wrapped Cryptopunks,sell,80.00
3,2021-08-06 18:16:51,Meebits,sell,15.00
4,2021-08-05 16:22:42,CryptoPunks,sell,99.00
...,...,...,...,...
3621,2021-07-04 08:13:26,Art Blocks,buy,0.04
3622,2021-07-04 07:20:27,BullsOnTheBlock,sell,0.20
3623,2021-07-04 07:01:36,Cool Cats,buy,0.04
3624,2021-07-04 06:59:58,Cool Cats,buy,0.04


In [98]:
top_buy_by_count = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "buy")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)
top_sell_by_count = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "sell")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)


In [110]:
top_buy_by_count

,asset_name,type,counts
2,First,buy,22
4,OpenSea Collection,buy,8
5,Role,buy,4
6,Sevens Token,buy,4
3,Lobby Lobsters,buy,2
7,SuperRare,buy,2
8,n,buy,2
0,Art Blocks,buy,1
1,CryptoPunks,buy,1


In [122]:
top_sell_by_count

,asset_name,type,counts
7,Gambling Apes,sell,14
2,CryptoKitties,sell,11
0,Art Blocks,sell,9
4,Degenz,sell,4
13,MonkeyBet,sell,4
20,Space Punks,sell,3
17,PudgyPresent,sell,3
11,MetaHero,sell,2
22,Void Attractors,sell,2
21,Stoner Cats,sell,2


In [230]:
top_buy_by_price = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "buy")
    ][["asset_name", "type", "price"]]
    .groupby(["asset_name", "type"])
    .sum()
    .sort_values("price", ascending=False)
)
top_sell_by_price = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "sell")
    ][["asset_name", "type", "price"]]
    .groupby(["asset_name", "type"])
    .sum()
    .sort_values("price", ascending=False)
)

In [302]:
top_buy_by_price

,,price
asset_name,type,
CryptoPunks,buy,119.0000
First,buy,40.0900
OpenSea Collection,buy,17.2400
Sevens Token,buy,9.2500
SuperRare,buy,8.5000
Art Blocks,buy,3.8000
n,buy,3.7498
Role,buy,1.0910
Lobby Lobsters,buy,0.0490


In [314]:
top_sell_by_price

,,price
asset_name,type,
Art Blocks,sell,94.400
MetaHero,sell,21.000
SALT,sell,15.000
Pulsquares,sell,13.000
PudgyPenguins,sell,11.690
MintPassFactory,sell,10.250
Cool Cats,sell,9.500
MutantApeYachtClub,sell,7.500
MonkeyBet,sell,5.948


In [ ]:
hist_dict = {'timestamp': [], 'asset_id': [], 'asset_name': [], 'token_id': [], 'seller': [], 'buyer': [], 'price': []}
for event in events:
    if event['asset'] is None:
        continue
    hist_dict['timestamp'].append(event['transaction']['timestamp'] if event['transaction'] else event['created_date'])
    hist_dict['asset_id'].append(event['asset']['id'])
    hist_dict['asset_name'].append(event['asset']['asset_contract']['name'])
    hist_dict['token_id'].append(event['asset']['token_id'])
    hist_dict['seller'].append(event['seller']['user']['username'] if event['seller']['user'] is not None else None)
    hist_dict['buyer'].append(event['winner_account']['user']['username'] if event['winner_account']['user'] is not None else None)
    hist_dict['price'].append(float(event['total_price']) / 10e17)

hist_df = pd.DataFrame.from_dict(hist_dict)

TypeError: 'NoneType' object is not subscriptable

price
asset_name    type          
$LONDON Gift  buy    0.13000
              sell   0.17000
AI42 Loops    sell   2.05000
AIFX          sell   3.50000
Ability Score sell   0.86500
...                      ...
n             buy    7.19680
              sell   8.29000
parallel      buy   65.21990
              sell  12.29500
sLoot         buy   23.27984

[223 rows x 1 columns]